# Compare All Models
## Chest X-ray Classification

Compare results from all 8 experiments (4 models × 2 loss functions).

In [ ]:
import sys
sys.path.append('..')

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import Image as IPImage, display

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Load All Results

In [ ]:
# Load summary CSV (generated by compare_results.py)
summary_path = Path('../results/summary.csv')

if summary_path.exists():
    df = pd.read_csv(summary_path)
    print("Loaded results for", len(df), "experiments")
    display(df)
else:
    print("Summary not found! Run: python scripts/experiments/compare_results.py")

## 2. Overall Ranking

In [ ]:
# Sort by accuracy
df_sorted = df.sort_values('accuracy', ascending=False)

print("\n🏆 MODEL RANKING BY ACCURACY:\n")
for i, row in enumerate(df_sorted.itertuples(), 1):
    print(f"{i}. {row.model} ({row.loss}): {row.accuracy:.2f}%")

## 3. Comparison Plots

In [ ]:
# Display saved comparison plots
plots = [
    '../results/accuracy_comparison.png',
    '../results/metrics_comparison.png',
    '../results/model_comparison_by_loss.png'
]

for plot_path in plots:
    if Path(plot_path).exists():
        print(f"\n{Path(plot_path).stem}:")
        display(IPImage(filename=plot_path))
    else:
        print(f"Plot not found: {plot_path}")

## 4. Loss Function Comparison

In [ ]:
# Compare Weighted CE vs Focal Loss
wce_mean = df[df['loss'] == 'Weighted CE']['accuracy'].mean()
focal_mean = df[df['loss'] == 'Focal Loss']['accuracy'].mean()

print("Average Performance by Loss Function:")
print(f"  Weighted CE:   {wce_mean:.2f}%")
print(f"  Focal Loss:    {focal_mean:.2f}%")
print(f"  Difference:    {abs(wce_mean - focal_mean):.2f}%")

winner = "Weighted CE" if wce_mean > focal_mean else "Focal Loss"
print(f"\n🏆 Better Loss: {winner}")

## 5. Model Architecture Comparison

In [ ]:
# Compare model architectures (average across both losses)
model_avg = df.groupby('model')['accuracy'].mean().sort_values(ascending=False)

print("Average Performance by Model Architecture:")
for model, acc in model_avg.items():
    print(f"  {model:18s}: {acc:.2f}%")

print(f"\n🏆 Best Architecture: {model_avg.idxmax()}")

## 6. Statistical Analysis

In [ ]:
# Box plot of accuracy distribution
fig, ax = plt.subplots(figsize=(10, 6))

df.boxplot(column='accuracy', by='loss', ax=ax)
ax.set_xlabel('Loss Function', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Accuracy Distribution by Loss Function', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

## 7. Best Model Summary

In [ ]:
# Get best overall model
best_row = df_sorted.iloc[0]

print("="*60)
print("🏆 BEST MODEL SUMMARY")
print("="*60)
print(f"Model:         {best_row['model']}")
print(f"Loss:          {best_row['loss']}")
print(f"Experiment ID: {best_row['experiment_id']}")
print("\nPerformance:")
print(f"  Accuracy:    {best_row['accuracy']:.2f}%")
print(f"  Precision:   {best_row['precision']:.2f}%")
print(f"  Recall:      {best_row['recall']:.2f}%")
print(f"  F1 Score:    {best_row['f1']:.2f}%")
if not pd.isna(best_row['auc']):
    print(f"  AUC:         {best_row['auc']:.2f}%")
print("="*60)

## 8. Export Results

In [ ]:
# Export formatted table
export_df = df_sorted[['model', 'loss', 'accuracy', 'precision', 'recall', 'f1']].copy()
export_df.columns = ['Model', 'Loss', 'Accuracy (%)', 'Precision (%)', 'Recall (%)', 'F1 (%)']
export_df = export_df.round(2)

print("\nFormatted Results Table:")
display(export_df)

# Save to LaTeX (for paper/thesis)
latex_path = '../results/results_table.tex'
export_df.to_latex(latex_path, index=False)
print(f"\n✓ LaTeX table saved to: {latex_path}")